In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import axes3d  
import seaborn as sns
from scipy import stats

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA

from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn import svm
import random
from imblearn.over_sampling import SMOTE
import rcca
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.feature_selection import RFE

In [3]:
from EDA_helper import *

In [15]:
ratings = pd.read_excel("../datasets/ratings_2_06-17.xlsx")
ratio = pd.read_excel("../datasets/ratio_figs_2.xlsx")
ratio = ratio.rename(columns = {'Public Date':'Data Date','EXCHANGE TICKER SYMBOL - HISTORICAL':'Ticker Symbol'})

###########################################################################################################
ratio1, ratings1 = format_df(ratio.copy(), ratings.copy(), ['Trailing P/E to Growth (PEG) ratio','Dividend Yield'])
numer = list(ratio1.columns[4:-1])
X1,Y1,full1 = join_numerical(ratio1, ratings1, numerical=numer)

ratio2, ratings2 = format_df(ratio.copy(), ratings.copy(), ['Trailing P/E to Growth (PEG) ratio','Dividend Yield', 
                        'Cash Ratio', 'Total Debt/Capital', 'Operating Profit Margin Before Depreciation',
                            'Enterprise Value Multiple', 'Cash Balance/Total Liabilities'])
numer = list(ratio2.columns[4:-1])
X2,Y2,full2 = join_numerical(ratio2, ratings2, numerical=numer)

ratio3, ratings3 = format_df(ratio.copy(), ratings.copy(), ['Trailing P/E to Growth (PEG) ratio','Dividend Yield', 
                        'Cash Ratio', 'Total Debt/Capital', 'Operating Profit Margin Before Depreciation',
                            'Enterprise Value Multiple', 'Cash Balance/Total Liabilities',
                            'Profit Before Depreciation/Current Liabilities', 'Cash Flow Margin'])
numer = list(ratio3.columns[4:-1])
X3,Y3,full3 = join_numerical(ratio3, ratings3, numerical=numer)

# ratio1, ratings1 = format_df(ratio.copy(), ratings.copy(), ['Trailing P/E to Growth (PEG) ratio','Dividend Yield'])
# numer = list(ratio1.columns[4:-1])
# X1,Y1,full1 = join_numerical(ratio1, ratings1, numerical=numer)


Unique Companies in Ratings:  88
Unique Companies in Features:  42
Intersection of companies:  26
Unique Companies in Ratings:  88
Unique Companies in Features:  42
Intersection of companies:  26
Unique Companies in Ratings:  88
Unique Companies in Features:  42
Intersection of companies:  26


In [16]:
# ac = pd.DataFrame(X,columns = numer)
# cor = ac.corr()
# plt.figure(figsize=(20,20))
# sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
# plt.show()


In [17]:
xs = [StandardScaler().fit_transform(X1), StandardScaler().fit_transform(X2), 
      StandardScaler().fit_transform(X3)]
ys = [encode(Y1)[0], encode(Y2)[0], encode(Y3)[0]]
fulls = [full1, full2, full3]

In [28]:
def GDA(X, Y, full, shrink=None, reg=0.0):
    
    results = {}
    X_train, Y_train, _, _, X_test, Y_test = data_split(X, Y, full, split=[0.8, 0.0],smote=True)
    # LDA w/ smote
    lda = LinearDiscriminantAnalysis(solver='eigen', shrinkage=shrink)
    lda.fit(X_train, Y_train)
    preds_train = lda.predict(X_train)
    preds_test = lda.predict(X_test)
    results['lda_smote'] = (np.mean(Y_train == preds_train), np.mean(Y_test == preds_test))
    # QDA w/ smote
    qda = QuadraticDiscriminantAnalysis(reg_param = reg)
    qda.fit(X_train, Y_train)
    preds_train = qda.predict(X_train)
    preds_test = qda.predict(X_test)
    
    results['qda_smote'] = (np.mean(Y_train == preds_train), np.mean(Y_test == preds_test))
    
    X_train, Y_train, _, _, X_test, Y_test = data_split(X, Y, full, split=[0.8, 0.0], smote=False)
    # LDA w/o smote
    lda = LinearDiscriminantAnalysis(solver='eigen',shrinkage=shrink)
    lda.fit(X_train, Y_train)
    preds_train = lda.predict(X_train)
    preds_test = lda.predict(X_test)
    results['lda'] = (np.mean(Y_train == preds_train), np.mean(Y_test == preds_test))
    # QDA w/o smote
    qda = QuadraticDiscriminantAnalysis(reg_param = reg)
    qda.fit(X_train, Y_train)
    preds_train = qda.predict(X_train)
    preds_test = qda.predict(X_test)
    results['qda'] = (np.mean(Y_train == preds_train), np.mean(Y_test == preds_test))    
    
    df_ = pd.DataFrame(results.values(),index=results.keys(), columns = ['train accuracy', 'test accuracy'])
    max_test=np.argmax(df_.iloc[:, 1:].values)
    max_elem = pd.DataFrame(df_.iloc[max_test, :])
    return df_, max_elem
    
    

In [29]:
 
dfs = []
maxis = []
for i in range(len(xs)):
    df, maxi = GDA(xs[i], ys[i], fulls[i])
    dfs.append(df)
    maxis.append(maxi)
pd.concat(dfs, axis=1)


c:\users\rodri\appdata\local\programs\python\python37\lib\site-packages\sklearn\discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\users\rodri\appdata\local\programs\python\python37\lib\site-packages\sklearn\discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\users\rodri\appdata\local\programs\python\python37\lib\site-packages\sklearn\discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\users\rodri\appdata\local\programs\python\python37\lib\site-packages\sklearn\discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\users\rodri\appdata\local\programs\python\python37\lib\site-packages\sklearn\discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\users\rodri\appdata\local\programs\python\pytho

,train accuracy,test accuracy,train accuracy,test accuracy,train accuracy,test accuracy
lda_smote,0.817300,0.511905,0.690717,0.482143,0.677637,0.482143
qda_smote,0.957806,0.744048,0.931224,0.696429,0.925316,0.678571
lda,0.764179,0.732143,0.720896,0.630952,0.704478,0.583333
qda,0.886567,0.791667,0.849254,0.761905,0.841791,0.744048


In [31]:
dfs = []
for i in [0.0, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0, 10.0]:
    df, maxi = GDA(xs[0], ys[0], fulls[0], shrink='auto', reg=i)
    dfs.append(df)
pd.concat(dfs, axis=1)

c:\users\rodri\appdata\local\programs\python\python37\lib\site-packages\sklearn\discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\users\rodri\appdata\local\programs\python\python37\lib\site-packages\sklearn\discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\users\rodri\appdata\local\programs\python\python37\lib\site-packages\sklearn\discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\users\rodri\appdata\local\programs\python\python37\lib\site-packages\sklearn\discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\users\rodri\appdata\local\programs\python\python37\lib\site-packages\sklearn\discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\users\rodri\appdata\local\programs\python\pytho

,train accuracy,test accuracy,train accuracy,test accuracy,train accuracy,test accuracy,train accuracy,test accuracy,train accuracy,test accuracy,train accuracy,test accuracy,train accuracy,test accuracy,train accuracy,test accuracy
lda_smote,0.776793,0.535714,0.776793,0.535714,0.776793,0.535714,0.776793,0.535714,0.776793,0.535714,0.776793,0.535714,0.776793,0.535714,0.776793,0.535714
qda_smote,0.957806,0.744048,0.957384,0.791667,0.956962,0.821429,0.954430,0.821429,0.927426,0.773810,0.841772,0.571429,0.662869,0.494048,0.166667,0.029762
lda,0.726866,0.642857,0.726866,0.642857,0.726866,0.642857,0.726866,0.642857,0.726866,0.642857,0.726866,0.642857,0.726866,0.642857,0.726866,0.642857
qda,0.886567,0.791667,0.885075,0.827381,0.879104,0.845238,0.874627,0.833333,0.819403,0.767857,0.685075,0.607143,0.680597,0.607143,0.014925,0.029762


### LDA w/ Smote

In [17]:
X_train, Y_train, X_val, Y_val, X_test, Y_test = data_split(scl_X, enc_Y, full, split=[0.8, 0.0],smote=True)

In [18]:
## TRAIN ACCURACY
lda = LinearDiscriminantAnalysis()

lda.fit(X_train, Y_train)
# preds_test = lda.predict(X_test)
preds_train = lda.predict(X_train)
print(classification_report(Y_train, preds_train))
# print(classification_report(Y_test, preds_test))
np.mean(Y_train == preds_train)
# np.mean(Y_test == preds_test)

              precision    recall  f1-score   support

           1       0.91      0.92      0.91       395
           2       0.68      0.64      0.66       395
           3       0.73      0.52      0.61       395
           4       0.74      0.91      0.81       395
           5       0.92      0.96      0.94       395
           6       0.89      0.96      0.92       395

    accuracy                           0.82      2370
   macro avg       0.81      0.82      0.81      2370
weighted avg       0.81      0.82      0.81      2370



0.8172995780590717

In [15]:
X_train, Y_train, X_val, Y_val, X_test, Y_test = data_split(scl_X, enc_Y, full, split=[0.7, 0.1],smote=True)

lda = LinearDiscriminantAnalysis()

lda.fit(X_train, Y_train)
# preds_test = lda.predict(X_test)
preds_train = lda.predict(X_val)
print(classification_report(Y_val, preds_train))
# print(classification_report(Y_test, preds_test))
np.mean(Y_val == preds_train)
pd.Series(Y_val).value_counts()

              precision    recall  f1-score   support

           1       0.22      0.67      0.33         3
           2       0.61      0.55      0.58        20
           3       0.82      0.28      0.42        50
           4       0.20      1.00      0.33         3
           5       0.45      1.00      0.62         5
           6       0.14      0.67      0.24         3

    accuracy                           0.44        84
   macro avg       0.41      0.69      0.42        84
weighted avg       0.68      0.44      0.46        84



3    50
2    20
5     5
6     3
4     3
1     3
dtype: int64

In [158]:
## VALIDATION ACCURACY USING CV
## Getting abnormally high accuracy compared to Test bc DISTRIBUTION is different
## CV is using Train data that has SMOTE, so has different dist. of Y compared to test
## the vals have same dist. as train when doing CV on the train itself
lda = LinearDiscriminantAnalysis()
cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=3, random_state=1)
scores = cross_val_score(lda, X_train, Y_train, scoring='accuracy', cv=cv, n_jobs=-1)
scores1 = cross_val_score(lda, X_train, Y_train, scoring='accuracy', n_jobs=-1)
# scores1 = cross_val_score(qda, scl_x, Y_emb, scoring='accuracy', cv=cv, n_jobs=-1)
# summarize result
print('LDA Mean Accuracy (Stratified): %.3f (%.3f)' % (np.mean(scores), np.std(scores)))
print('LDA Mean Accuracy: %.3f (%.3f)' % (np.mean(scores1), np.std(scores1)))

# print('QDA Mean Accuracy: %.3f (%.3f)' % (np.mean(scores1), np.std(scores1)))

LDA Mean Accuracy (Stratified): 0.811 (0.010)
LDA Mean Accuracy: 0.809 (0.007)


In [19]:
## TEST ACCURACY
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, Y_train)
preds_test = lda.predict(X_test)
print(classification_report(Y_test, preds_test))
np.mean(Y_test == preds_test)

              precision    recall  f1-score   support

           1       0.19      0.60      0.29         5
           2       0.45      0.60      0.51        35
           3       0.82      0.42      0.55       101
           4       0.24      0.75      0.36         8
           5       0.64      0.88      0.74         8
           6       0.39      0.64      0.48        11

    accuracy                           0.51       168
   macro avg       0.45      0.65      0.49       168
weighted avg       0.66      0.51      0.53       168



0.5119047619047619

### LDA w/o SMOTE

In [135]:
X_train, Y_train, X_val, Y_val, X_test, Y_test = data_split(scl_X, enc_Y, full, split=[0.8, 0.0],smote=False, seed=200)

In [136]:
## TRAIN ACCURACY
lda = LinearDiscriminantAnalysis()

lda.fit(X_train, Y_train)
# preds_test = lda.predict(X_test)
preds_train = lda.predict(X_train)
print(classification_report(Y_train, preds_train))
# print(classification_report(Y_test, preds_test))
np.mean(Y_train == preds_train)
# np.mean(Y_test == preds_test)

              precision    recall  f1-score   support

           1       0.57      0.73      0.64        11
           2       0.71      0.46      0.56       153
           3       0.81      0.91      0.85       405
           4       0.47      0.48      0.47        29
           5       0.72      0.92      0.81        37
           6       0.88      0.60      0.71        35

    accuracy                           0.77       670
   macro avg       0.69      0.68      0.68       670
weighted avg       0.76      0.77      0.76       670



0.7686567164179104

In [ ]:
## Raw Validation w/o CV
# X_train, Y_train, X_val, Y_val, X_test, Y_test = data_split(scl_X, enc_Y, full, split=[0.7, 0.1],smote=True)

# lda = LinearDiscriminantAnalysis()

# lda.fit(X_train, Y_train)
# # preds_test = lda.predict(X_test)
# preds_train = lda.predict(X_val)
# print(classification_report(Y_val, preds_train))
# # print(classification_report(Y_test, preds_test))
# np.mean(Y_val == preds_train)
# pd.Series(Y_val).value_counts()

In [157]:
## VALIDATION ACCURACY USING CV

lda = LinearDiscriminantAnalysis()
cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=3, random_state=1)
scores = cross_val_score(lda, X_train, Y_train, scoring='accuracy', cv=cv, n_jobs=-1)
scores1 = cross_val_score(lda, X_train, Y_train, scoring='accuracy', n_jobs=-1)
# scores1 = cross_val_score(qda, scl_x, Y_emb, scoring='accuracy', cv=cv, n_jobs=-1)
# summarize result
print('LDA Mean Accuracy (Stratified): %.3f (%.3f)' % (np.mean(scores), np.std(scores)))
print('LDA Mean Accuracy: %.3f (%.3f)' % (np.mean(scores1), np.std(scores1)))

# print('QDA Mean Accuracy: %.3f (%.3f)' % (np.mean(scores1), np.std(scores1)))

LDA Mean Accuracy (Stratified): 0.811 (0.010)
LDA Mean Accuracy: 0.809 (0.007)


In [138]:
## TEST ACCURACY
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, Y_train)
preds_test = lda.predict(X_test)
print(classification_report(Y_test, preds_test))
np.mean(Y_test == preds_test)

              precision    recall  f1-score   support

           1       0.20      0.25      0.22         4
           2       0.67      0.47      0.55        43
           3       0.76      0.90      0.82        91
           4       0.50      0.36      0.42        14
           5       1.00      1.00      1.00        10
           6       1.00      0.83      0.91         6

    accuracy                           0.73       168
   macro avg       0.69      0.63      0.65       168
weighted avg       0.72      0.73      0.72       168



0.7321428571428571

### QDA w/ SMOTE

In [140]:
X_train, Y_train, X_val, Y_val, X_test, Y_test = data_split(scl_X, enc_Y, full, split=[0.8, 0.0],smote=True)

In [153]:
qda = QuadraticDiscriminantAnalysis()

qda.fit(X_train, Y_train)

preds_train = qda.predict(X_train)
print(classification_report(Y_train, preds_train))
np.mean(Y_train == preds_train)


              precision    recall  f1-score   support

           1       1.00      1.00      1.00       395
           2       0.86      0.90      0.88       395
           3       0.89      0.86      0.87       395
           4       1.00      0.99      1.00       395
           5       1.00      1.00      1.00       395
           6       1.00      1.00      1.00       395

    accuracy                           0.96      2370
   macro avg       0.96      0.96      0.96      2370
weighted avg       0.96      0.96      0.96      2370



c:\users\rodri\appdata\local\programs\python\python37\lib\site-packages\sklearn\discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


0.9578059071729957

In [156]:
## VALIDATION ACCURACY USING CV
qda_cv = QuadraticDiscriminantAnalysis()
cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=3, random_state=1)
scores = cross_val_score(qda_cv, X_train, Y_train, scoring='accuracy', cv=cv, n_jobs=-1)
scores1 = cross_val_score(qda_cv, X_train, Y_train, scoring='accuracy', n_jobs=-1)
# scores1 = cross_val_score(qda, scl_x, Y_emb, scoring='accuracy', cv=cv, n_jobs=-1)
# summarize result
print('QDA Mean Accuracy (Stratified): %.3f (%.3f)' % (np.mean(scores), np.std(scores)))
print('QDA Mean Accuracy: %.3f (%.3f)' % (np.mean(scores1), np.std(scores1)))

# print('QDA Mean Accuracy: %.3f (%.3f)' % (np.mean(scores1), np.std(scores1)))

QDA Mean Accuracy (Stratified): 0.944 (0.012)
QDA Mean Accuracy: 0.942 (0.017)


In [154]:
preds_test = qda.predict(X_test)
print(classification_report(Y_test, preds_test))
np.mean(Y_test == preds_test)

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         5
           2       0.60      0.77      0.68        35
           3       0.77      0.84      0.81       101
           4       1.00      0.62      0.77         8
           5       1.00      0.25      0.40         8
           6       1.00      0.55      0.71        11

    accuracy                           0.74       168
   macro avg       0.73      0.51      0.56       168
weighted avg       0.75      0.74      0.73       168



c:\users\rodri\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.7440476190476191

In [7]:
X_train, Y_train, X_val, Y_val, X_test, Y_test = data_split(scl_X, enc_Y, full, split=[0.8, 0.0])

In [11]:
qda = QuadraticDiscriminantAnalysis()

qda.fit(X_train, Y_train)

preds_train = qda.predict(X_train)
print(classification_report(Y_train, preds_train))
np.mean(Y_train == preds_train)

              precision    recall  f1-score   support

           1       1.00      1.00      1.00        10
           2       0.78      0.73      0.76       161
           3       0.89      0.92      0.91       395
           4       1.00      1.00      1.00        35
           5       1.00      1.00      1.00        39
           6       1.00      1.00      1.00        30

    accuracy                           0.89       670
   macro avg       0.95      0.94      0.94       670
weighted avg       0.88      0.89      0.89       670



c:\users\rodri\appdata\local\programs\python\python37\lib\site-packages\sklearn\discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


0.8865671641791045

In [12]:
## VALIDATION ACCURACY USING CV
qda_cv = QuadraticDiscriminantAnalysis()
cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=3, random_state=1)
scores = cross_val_score(qda_cv, X_train, Y_train, scoring='accuracy', cv=cv, n_jobs=-1)
scores1 = cross_val_score(qda_cv, X_train, Y_train, scoring='accuracy', n_jobs=-1)
# scores1 = cross_val_score(qda, scl_x, Y_emb, scoring='accuracy', cv=cv, n_jobs=-1)
# summarize result
print('QDA Mean Accuracy (Stratified): %.3f (%.3f)' % (np.mean(scores), np.std(scores)))
print('QDA Mean Accuracy: %.3f (%.3f)' % (np.mean(scores1), np.std(scores1)))

# print('QDA Mean Accuracy: %.3f (%.3f)' % (np.mean(scores1), np.std(scores1)))

QDA Mean Accuracy (Stratified): 0.720 (0.031)
QDA Mean Accuracy: 0.778 (0.043)


In [13]:
preds_test = qda.predict(X_test)
print(classification_report(Y_test, preds_test))
np.mean(Y_test == preds_test)

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         5
           2       0.73      0.54      0.62        35
           3       0.77      0.95      0.85       101
           4       1.00      0.75      0.86         8
           5       1.00      0.62      0.77         8
           6       1.00      0.64      0.78        11

    accuracy                           0.79       168
   macro avg       0.75      0.58      0.65       168
weighted avg       0.78      0.79      0.77       168



c:\users\rodri\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.7916666666666666

In [8]:


# evaluate model
scores = cross_val_score(lda, cca_df[['CVX1', 'CVX2']], cca_df['Y'], scoring='accuracy', cv=cv, n_jobs=-1)
scores1 = cross_val_score(qda, cca_df[['CVX1', 'CVX2']], cca_df['Y'], scoring='accuracy', cv=cv, n_jobs=-1)

print('LDA Mean Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))
print('QDA Mean Accuracy: %.3f (%.3f)' % (np.mean(scores1), np.std(scores1)))

scores = cross_val_score(lda, scl_x, Y_emb, scoring='accuracy', cv=cv, n_jobs=-1)
scores1 = cross_val_score(qda, scl_x, Y_emb, scoring='accuracy', cv=cv, n_jobs=-1)
# summarize result
print('LDA Mean Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))
print('QDA Mean Accuracy: %.3f (%.3f)' % (np.mean(scores1), np.std(scores1)))

scores = cross_val_score(lda, smote_x, smote_y, scoring='accuracy', cv=cv, n_jobs=-1)
scores1 = cross_val_score(qda, smote_x, smote_y, scoring='accuracy', cv=cv, n_jobs=-1)
# summarize result
print('LDA Mean Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))
print('QDA Mean Accuracy: %.3f (%.3f)' % (np.mean(scores1), np.std(scores1)))

NameError: name 'cca_df' is not defined

In [ ]:
qda = QuadraticDiscriminantAnalysis()